In [ ]:
pip install icalendar pytz

In [ ]:
import csv
import os
from datetime import datetime, timedelta
import pytz
from icalendar import Calendar, Event, Timezone, TimezoneStandard, TimezoneDaylight
import uuid

def read_csv(file_path):
    data = []
    with open(file_path, mode='r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file, delimiter=';')
        header = next(csv_reader)
        for row in csv_reader:
            if len(row) == 6:
                datum, tag, zeit, thema, ort, zuständig = row
                data.append({
                    'Datum': datum,
                    'Tag': tag,
                    'Uhrzeit': zeit,
                    'Thema': thema,
                    'Ort': ort,
                    'Zuständig': zuständig
                })
    return data

def create_event(name, start, end, description, timezone='Europe/Berlin'):
    event = Event()
    event.add('summary', name)
    event.add('dtstart', datetime.strptime(start, '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.timezone(timezone)))
    event.add('dtend', datetime.strptime(end, '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.timezone(timezone)))
    event.add('description', description)
    event.add('dtstamp', datetime.now(pytz.timezone(timezone)))
    event.add('uid', str(uuid.uuid4()))
    return event

def add_timezone(calendar, timezone='Europe/Berlin'):
    tz = Timezone()
    tz.add('tzid', timezone)
    tz.add('x-lic-location', timezone)

    standard = TimezoneStandard()
    standard.add('tzname', 'CET')
    standard.add('dtstart', datetime(1970, 10, 25, 3, 0, 0))
    standard.add('rrule', {'freq': 'yearly', 'bymonth': 10, 'byday': '-1su'})
    standard.add('tzoffsetfrom', timedelta(hours=2))
    standard.add('tzoffsetto', timedelta(hours=1))

    daylight = TimezoneDaylight()
    daylight.add('tzname', 'CEST')
    daylight.add('dtstart', datetime(1970, 3, 29, 2, 0, 0))
    daylight.add('rrule', {'freq': 'yearly', 'bymonth': 3, 'byday': '-1su'})
    daylight.add('tzoffsetfrom', timedelta(hours=1))
    daylight.add('tzoffsetto', timedelta(hours=2))

    tz.add_component(standard)
    tz.add_component(daylight)
    calendar.add_component(tz)


# Example usage
csv_data = read_csv(file_path)

# Create a new calendar
calendar = Calendar()
calendar.add('prodid', 'FFW BLB Abt Asch')
calendar.add('version', '2.0')

# Add timezone information
add_timezone(calendar)

# Loop through the CSV data and create events
for row in csv_data:
    # Add year 2025 to the date
    start_date = f"2025-{row['Datum'].split('.')[1]}-{row['Datum'].split('.')[0]}"
    start_time = row['Uhrzeit'].replace(' Uhr', ':00')
    start = f"{start_date} {start_time}"
    
    # Assuming the event lasts 1 hour for the end time
    end_datetime = datetime.strptime(f"{start_date} {start_time}", '%Y-%m-%d %H:%M:%S') + timedelta(hours=1, minutes=30)
    end = end_datetime.strftime('%Y-%m-%d %H:%M:%S')
    
    event = create_event(row['Thema'], start, end, f"Ort: {row['Ort']}, Zuständig: {row['Zuständig']}")
    calendar.add_component(event)

# Write the calendar to an ICS file
ics_filename = os.path.splitext(file_path)[0] + '.ics'

with open(ics_filename, 'wb') as f:
    f.write(calendar.to_ical())

print("ICS file generated successfully.")